<a href="https://colab.research.google.com/github/Kang-Beom-Seo/Tensorflow_tutorial/blob/main/(%EC%B4%88%EB%B3%B4%EC%9E%90)%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EB%A5%BC%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
print("훈련 샘플: {}, 레이블: {}".format(len(train_data), len(train_labels)))

In [ ]:
print(train_data[0])

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:
# 단어와 정수 인덱스를 매핑한 딕셔너리
word_index = imdb.get_word_index()

In [ ]:
print(word_index) #key값이 영단어이고, value값은 정수인걸로 보인다.

In [ ]:
print(word_index.items()) # key와 value쌍이 각각 튜플로 묶이는 개체로, 이 개체를 통해서 for문을 돌릴 수 있다. 

In [ ]:
word_index = {k:(v+3) for k,v in word_index.items()} #k는 key, v는 value로 뽑혀나온 값이 k:(v+3)의 형태의 쌍으로 다시 딕셔너리 안에 들어간다.'
                                                     #즉 원래 딕셔너리에서 value들이 원래값보다 +3된 결과이다.

In [ ]:
print(word_index) #아까 위의 행의 딕셔너리와 비교결과, key는 그대로고 value들만 3씩 올랐다.

In [ ]:
word_index["<PAD>"] = 0 #Padding 토큰으로, 가변적인 입력길이를 맞추려고 넣는 것이다. 값은 0
word_index["<START>"] = 1 #Start 토큰으로, 해당 문장이 시작된다는 것을 알려주는 것이다. 값은 1
word_index["<UNK>"] = 2  # unknown 토큰으로, 희귀한 단어들을 의미하는 것이다. 값은 2
word_index["<UNUSED>"] = 3 #unused 토큰으로, unknown이 있는데 이 토큰이 왜 있는지는 아직 모르겠다. 값은 3
                         # 이들을 살펴보면 결국 토큰 값이 3까지 있으며, 이 토큰들과 값을 안겹치게 하기 위해 딕셔너리의 value들을 3씩 더한것으로 보인다.

In [ ]:
word_index["<PAD>"]

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()]) #딕셔너리의 key와 value를 뒤집는다. 즉, 이제 숫자로 단어를 조회하는게 가능해졌다.

In [ ]:
print(reverse_word_index)

In [ ]:
print(reverse_word_index[4]) #임의의 숫자를 넣으면 영단어가 잘 조회된다.

In [ ]:
def decode_review(text): #숫자로 입력된 문장을 입력받으면, 위의 딕셔너리에서 숫자를 넣어 단어들을 조회해서 문장으로 만들어주는 함수이다.
    return ' '.join([reverse_word_index.get(i, '?') for i in text]) #get(i, '?')에서 '?'는 해당 단어를 못찾으면 '?'로 리턴하는 것이다.

In [ ]:
decode_review(train_data[0]) #이와 같이 잘 출력된다. 

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, #여기에는 list가 들어온다. list의 길이는 훈련시킬 데이터의 개수이다.
                                                        value=word_index["<PAD>"], #value인수에는 어떤 값으로 패딩할지가 들어가며, 여기서는 0이 들어간다.
                                                        padding='post', #이거는 패딩하는 방식으로, pre는 앞에서부터 채우는거고, post는 뒤를 채우는거다.
                                                        maxlen=256) #패딩해서 나오는 최종적인 길이다.

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256) #위와 동일한 방식으로 test_data도 패딩을 한다.

In [ ]:
print(len(train_data[0]), len(train_data[1])) #길이가 256으로 잘 패딩되었다.

In [ ]:
print(train_data[0]) #뒤에는 잘 0으로 패딩되었다.

# **데이터 전처리가 끝나고, 이제부터는 모델을 구성하는 부분입니다.**




In [ ]:
vocab_size = 10000 #단어사전 크기가 10000이여서 이렇게 지정함.

In [ ]:
model = keras.Sequential() #이 모델은 정확히 1개의 입력텐서, 1개의 출력텐서가 있어야 적합하다.
                           #이 모델의 경우, 문장을 받고, 출력의 경우 부정이나 긍정(0 또는 1)이 나오므로 이 모델을 쓰기 적합하다.

In [ ]:
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,))) #이 레이어는 워드 임베딩을 담당하는 레이어로, 이 레이어를 거치면
                                                                       # (데이터수, 문장 길이, 임베딩된 차원)의 형태로 리턴된다.

In [ ]:
model.add(keras.layers.GlobalAveragePooling1D()) #pooling 레이어로, pooling은 가중치들의 차원이 너무 커지는 걸 막기위해, 값을 선별하여 추린다고 생각하면 된다.
                                                 #이 레이어 인수에는 data_format이 있는데, default값은 channels_last다. 이는 입력 텐서의 shape이
                                                 #(batch, steps, features)일때 사용하며, channels_first는 (batch, features, steps)일때 사용한다.
                                                 #여기서는 문장길이가 steps이며, 임베딩 차원이 features이므로 last방식을 이용하여 pooling하는 것이다.

In [ ]:
model.add(keras.layers.Dense(16, activation='relu')) #fully-connected layer로, 활성함수는 relu로 사용하였다. relu는 max(0,x)로 음수는 0, 양수는 값을 그대로 쓰는 함수다.

In [ ]:
model.add(keras.layers.Dense(1, activation='sigmoid')) #최종 layer로 sigmoid를 통해 값이 0~1로 결정된다.

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', #adam 최적화 알고리즘을 사용한다. 자세한 알고리즘은 너무 복잡해서 나중에 봐야할듯 하다.
              loss='binary_crossentropy', # 손실함수로는 binary_cross_entropy를 사용하였으며, cross_entropy중에서도 두개의 class(긍정, 부정)중 하나를 예측하는 binary로 사용하였다.
              metrics=['accuracy']) # 정확도를 평가척도로 사용한다.

In [ ]:
x_val = train_data[:10000] #validation 세트
partial_x_train = train_data[10000:] #나머지 훈련세트

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

In [ ]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1) #acc에는 데이터 배치 하나당 하나씩의 accuracy가 있으므로, acc의 길이가 곧 데이터 배치의 총개수가 된다.

In [ ]:
# "bo"는 "파란색 점"입니다
plt.plot(epochs, loss, 'bo', label='Training loss') #왼쪽에서부터 x축, y축, 색깔, label이다.
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend() #범례(위의 label)를 표시한다.

plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()